In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

In [2]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [4]:
# read in the river profile CSV
data_dir = '/media/TopographicData/TopographicData/san_andreas/SAF_combined/SAF_only/'
df = pd.read_csv(data_dir+'SAF_only_profiles_fault_dist_SO3.csv')
df = df[df['slope'] > 0]
df.columns

Index(['basin_id', 'id', 'node', 'distance_from_outlet', 'elevation',
       'drainage_area', 'stream_order', 'slope', 'latitude', 'longitude',
       'new_id', 'fault_dist', 'direction'],
      dtype='object')

In [5]:
# read in the hillslope metrics CSV
hs_df = pd.read_csv(data_dir+'SAF_only_hillslopes_SO3.csv')

In [6]:
hs_df

,basin_id,slope_median,slope_16th,slope_84th,latitude,longitude,new_id
0,216,0.357944,0.251787,0.568092,36.207419,-120.767752,0.0
1,287,0.224481,0.144274,0.365453,36.207421,-120.771712,1.0
2,366,0.186452,0.075103,0.370029,36.205171,-120.769484,2.0
3,397,0.252137,0.199470,0.330081,36.205171,-120.769473,3.0
4,460,0.393487,0.203220,0.629382,36.203571,-120.763446,4.0
...,...,...,...,...,...,...,...
16106,9463,0.579460,0.238936,0.903064,33.613601,-116.031564,40299.0
16107,9478,0.285763,0.117563,0.569832,33.612765,-116.036489,40300.0
16108,9522,0.499455,0.225669,0.754292,33.614905,-116.037964,40301.0
16109,9561,0.287724,0.104336,0.558448,33.612141,-116.034028,40302.0


In [7]:
# convert the river csv to a geodataframe. Remove the non-unique ID labels - these will be replaced by unique basin IDs
geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
crs = 'epsg:4326' #http://www.spatialreference.org/ref/epsg/2263/
river_gdf = gpd.GeoDataFrame(df.drop(['latitude','longitude','basin_id','id','new_id','node'], axis=1), crs=crs, geometry=geometry)

In [8]:
# convert the hillslope csv to a geodataframe. Remove the non-unique ID labels
geometry = [Point(xy) for xy in zip(hs_df.longitude, hs_df.latitude)]
hs_gdf = gpd.GeoDataFrame(hs_df.drop(['latitude','longitude','basin_id','new_id'], axis=1), crs=crs, geometry=geometry)

In [9]:
hs_gdf

,slope_median,slope_16th,slope_84th,geometry
0,0.357944,0.251787,0.568092,POINT (-120.76775 36.20742)
1,0.224481,0.144274,0.365453,POINT (-120.77171 36.20742)
2,0.186452,0.075103,0.370029,POINT (-120.76948 36.20517)
3,0.252137,0.199470,0.330081,POINT (-120.76947 36.20517)
4,0.393487,0.203220,0.629382,POINT (-120.76345 36.20357)
...,...,...,...,...
16106,0.579460,0.238936,0.903064,POINT (-116.03156 33.61360)
16107,0.285763,0.117563,0.569832,POINT (-116.03649 33.61276)
16108,0.499455,0.225669,0.754292,POINT (-116.03796 33.61490)
16109,0.287724,0.104336,0.558448,POINT (-116.03403 33.61214)


In [10]:
# add a unique id to the basin
basin_gdf = gpd.read_file(data_dir+'SAF_only_basins_deflection.shp')
basin_gdf = basin_gdf.drop(['basin_id'], axis=1)
basin_gdf['unique_id'] = basin_gdf.index

In [11]:
river_gdf

,distance_from_outlet,elevation,drainage_area,stream_order,slope,fault_dist,direction,geometry
0,472.956000,754.330000,1443,1,0.280238,524.210347,-0.000194,POINT (-120.77122 36.20501)
1,471.541504,754.309998,1445,1,0.284092,524.210347,-0.000194,POINT (-120.77121 36.20502)
2,470.541504,753.969971,1446,1,0.288058,524.210347,-0.000194,POINT (-120.77120 36.20503)
3,469.541504,753.500000,1449,1,0.289001,524.210347,-0.000195,POINT (-120.77119 36.20503)
4,468.541504,752.979980,1450,1,0.289479,524.210347,-0.000195,POINT (-120.77118 36.20503)
...,...,...,...,...,...,...,...,...
5993315,3.414214,19.459999,56283,3,0.034965,1061.679334,0.000049,POINT (-116.03499 33.61016)
5993316,2.414214,19.430000,56300,3,0.034903,1061.679334,0.000049,POINT (-116.03500 33.61016)
5993317,1.000000,19.369999,56363,3,0.035012,1061.679334,0.000049,POINT (-116.03501 33.61016)
5993318,0.000000,19.330000,56364,3,0.034542,1061.679334,0.000049,POINT (-116.03502 33.61017)


In [12]:
basin_gdf

,basin_area,azimuth,deflection,latitude,longitude,fault_dist,direction,geometry,unique_id
0,1.040850e-10,19.419021,65.075406,39.021498,-123.698779,117.191340,-0.000335,"POLYGON ((-123.69989 39.01899, -123.69988 39.0...",0
1,5.548658e-06,7.964627,50.894860,39.021498,-123.698779,117.191340,-0.000335,"POLYGON ((-123.69878 39.02150, -123.69877 39.0...",1
2,1.887165e-06,304.337516,21.060099,39.019370,-123.709397,117.191340,-0.000131,"POLYGON ((-123.70842 39.02008, -123.70840 39.0...",2
3,1.040820e-10,359.822731,40.489295,39.019973,-123.703650,117.191340,-0.000233,"POLYGON ((-123.70362 39.01714, -123.70360 39.0...",3
4,7.280988e-06,346.132682,23.177698,39.019973,-123.703650,117.191340,-0.000233,"POLYGON ((-123.70366 39.01998, -123.70364 39.0...",4
...,...,...,...,...,...,...,...,...,...
21540,5.478528e-06,158.762914,28.661965,33.556380,-115.967462,1070.409964,-0.000002,None,21540
21541,5.549137e-06,156.197708,25.725476,33.556374,-115.967839,1070.409964,0.000003,None,21541
21542,2.212154e-06,143.018528,11.224846,33.556366,-115.967839,1070.409964,0.000003,None,21542
21543,3.021460e-06,159.512253,29.523176,33.554631,-115.971845,1070.409964,0.000088,None,21543


In [13]:
join = gpd.sjoin(river_gdf, basin_gdf, how='left', op='intersects')

In [14]:
gr = join.groupby(['unique_id'])['slope'].agg(['median', 'std', percentile(16), percentile(84)]).rename(columns={'median': 'channel_slope_median', 'std': 'channel_slope_std', 'percentile_16': 'channel_slope_16th', 'percentile_84': 'channel_slope_q2'}).reset_index()

In [15]:
basin_gdf = basin_gdf.merge(gr, on='unique_id')

In [16]:
basin_gdf

,basin_area,azimuth,deflection,latitude,longitude,fault_dist,direction,geometry,unique_id,channel_slope_median,channel_slope_std,channel_slope_16th,channel_slope_q2
0,5.548658e-06,7.964627,50.894860,39.021498,-123.698779,117.191340,-0.000335,"POLYGON ((-123.69878 39.02150, -123.69877 39.0...",1,0.000503,0.000016,0.000490,0.000517
1,1.887165e-06,304.337516,21.060099,39.019370,-123.709397,117.191340,-0.000131,"POLYGON ((-123.70842 39.02008, -123.70840 39.0...",2,0.000512,0.000450,0.000493,0.001102
2,7.280988e-06,346.132682,23.177698,39.019973,-123.703650,117.191340,-0.000233,"POLYGON ((-123.70366 39.01998, -123.70364 39.0...",4,0.000503,0.000034,0.000479,0.000537
3,7.925956e-06,314.561815,11.609073,39.019973,-123.703627,117.191340,-0.000234,"POLYGON ((-123.70208 39.02186, -123.70207 39.0...",5,0.000513,0.000781,0.000482,0.000584
4,2.133768e-06,87.725781,50.831639,39.016151,-123.694868,117.191340,-0.000322,"POLYGON ((-123.69568 39.01741, -123.69565 39.0...",7,0.000502,0.001032,0.000489,0.001981
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18042,3.289491e-06,125.925873,3.998710,33.932735,-116.459241,1007.734905,-0.000023,"POLYGON ((-116.46134 33.93553, -116.46132 33.9...",20603,0.022872,0.008700,0.017769,0.031185
18043,1.460510e-06,229.354705,62.701008,33.932268,-116.470064,1007.734905,0.000105,"POLYGON ((-116.46657 33.93504, -116.46655 33.9...",20604,0.028505,0.006440,0.020721,0.035488
18044,9.758000e-11,88.654369,28.652033,33.932268,-116.470064,1007.734905,0.000105,"POLYGON ((-116.47007 33.93227, -116.47006 33.9...",20605,0.030096,NaN,0.030096,0.030096
18045,1.844055e-06,179.439410,63.138660,33.930297,-116.462697,1007.734905,0.000060,"POLYGON ((-116.46319 33.93268, -116.46315 33.9...",20606,0.122196,0.137834,0.023186,0.308220


In [29]:
# now join the hillslope data
join = gpd.sjoin(basin_gdf, hs_gdf, how='left', op='contains')

In [30]:
join

,basin_area,azimuth,deflection,latitude,longitude,fault_dist,direction,geometry,unique_id,channel_slope_median,channel_slope_std,channel_slope_16th,channel_slope_q2,index_right,slope_median,slope_16th,slope_84th
0,5.548658e-06,7.964627,50.894860,39.021498,-123.698779,117.191340,-0.000335,"POLYGON ((-123.69878 39.02150, -123.69877 39.0...",1,0.000503,0.000016,0.000490,0.000517,2372.0,0.011662,0.000707,0.062896
1,1.887165e-06,304.337516,21.060099,39.019370,-123.709397,117.191340,-0.000131,"POLYGON ((-123.70842 39.02008, -123.70840 39.0...",2,0.000512,0.000450,0.000493,0.001102,2373.0,0.023138,0.000871,0.056654
2,7.280988e-06,346.132682,23.177698,39.019973,-123.703650,117.191340,-0.000233,"POLYGON ((-123.70366 39.01998, -123.70364 39.0...",4,0.000503,0.000034,0.000479,0.000537,2375.0,0.002597,0.000705,0.027232
3,7.925956e-06,314.561815,11.609073,39.019973,-123.703627,117.191340,-0.000234,"POLYGON ((-123.70208 39.02186, -123.70207 39.0...",5,0.000513,0.000781,0.000482,0.000584,2371.0,0.014578,0.000708,0.049056
4,2.133768e-06,87.725781,50.831639,39.016151,-123.694868,117.191340,-0.000322,"POLYGON ((-123.69568 39.01741, -123.69565 39.0...",7,0.000502,0.001032,0.000489,0.001981,2376.0,0.013362,0.000708,0.044648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18042,3.289491e-06,125.925873,3.998710,33.932735,-116.459241,1007.734905,-0.000023,"POLYGON ((-116.46134 33.93553, -116.46132 33.9...",20603,0.022872,0.008700,0.017769,0.031185,15177.0,0.036326,0.021324,0.143496
18043,1.460510e-06,229.354705,62.701008,33.932268,-116.470064,1007.734905,0.000105,"POLYGON ((-116.46657 33.93504, -116.46655 33.9...",20604,0.028505,0.006440,0.020721,0.035488,NaN,NaN,NaN,NaN
18044,9.758000e-11,88.654369,28.652033,33.932268,-116.470064,1007.734905,0.000105,"POLYGON ((-116.47007 33.93227, -116.47006 33.9...",20605,0.030096,NaN,0.030096,0.030096,15178.0,0.037938,0.026838,0.052327
18045,1.844055e-06,179.439410,63.138660,33.930297,-116.462697,1007.734905,0.000060,"POLYGON ((-116.46319 33.93268, -116.46315 33.9...",20606,0.122196,0.137834,0.023186,0.308220,15183.0,0.264323,0.077764,0.380089


In [31]:
len(join.unique_id.unique())

18047

In [32]:
join.to_file(data_dir+'SAF_only_channels_plus_hillslopes_by_basin_SO3.shp')

In [27]:
len(join)

18047